<a href="https://colab.research.google.com/github/adityaray7/DS201-Assignments/blob/main/assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# 1) Generate a sinusoidal signal. Add noise to it.

In [1]:
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import wave, sys

In [2]:
#using plotly to plot graphs
import plotly.io as plt_io

plt_io.templates["custom_dark"] = plt_io.templates["plotly_dark"]

plt_io.templates["custom_dark"]['layout']['paper_bgcolor'] = '#30404D'
plt_io.templates["custom_dark"]['layout']['plot_bgcolor'] = '#30404D'

plt_io.templates['custom_dark']['layout']['yaxis']['gridcolor'] = '#4f687d'
plt_io.templates['custom_dark']['layout']['xaxis']['gridcolor'] = '#4f687d'


In [3]:
# Set parameters for the sinosuidal signal
amplitude = 1
frequency = 2  # Hz
phase = 0  # radians

# Generate the time vector
time = np.linspace(0, 2*np.pi, 1000)

# Generate the sinusoidal signal
OriginalSignal = amplitude * np.sin(2*np.pi*frequency*time + phase)

In [4]:
# Add random noise to the signal
noise_amplitude = 0.2
def random_noise(signal,noise_amplitude):
  noise = noise_amplitude * np.random.randn(len(signal))
  NoisySignal = signal + noise
  return NoisySignal

# Add Gaussian Noise
mu=0.0
std = 0.1
def gaussian_noise(x,mu,std):
    noise_gaussian = np.random.normal(mu, std, size = x.shape)
    NoisySignal = x + noise_gaussian
    return NoisySignal

#two types of noises for sinusoidal
GaussianNoiseSignal = gaussian_noise(OriginalSignal,mu,std)
RandomNoiseSignal = random_noise(OriginalSignal,noise_amplitude)

In [5]:
first_line = go.Scatter(x=time, y=OriginalSignal, name="Original Signal")
second_line = go.Scatter(x= time, y= GaussianNoiseSignal, name="Gaussian Noise Signal")
third_line = go.Scatter(x= time, y= RandomNoiseSignal, name=" Random Noise Signal")

fig = make_subplots(rows=3, cols=1, vertical_spacing=0.10)

fig.add_trace(first_line, row=1, col=1)
fig.add_trace(second_line, row=2, col=1)
fig.add_trace(third_line, row=3, col=1)

fig.update_layout(height=600, width=800, title_text=" Signal Visualization")
fig.layout.template = 'custom_dark'
fig.show()

Perform convolutions with appropriate filters to reduce the noise.
(Make your code flexible so as to accept any signal and any filter) 

In [6]:
#creating two signals - toneburst and complete signal to test the pipeline against custom code
nocycles = 10
freq = 4*10**5
samplefreq = 4*10**7
def damage_toneburst(nocycles,freq,samplefreq ):
    T = nocycles/freq
    N = (nocycles/freq)*samplefreq
    n = np.arange(0,N,1)
    tone=np.sin((nocycles*2*np.pi*n)/N);
    burst=0.5*(1-np.cos((2*np.pi*n)/N));
    tb=tone*burst
    return tb

locs = [0,400,800]  #ranges in which toneburst activates
amps = [1,2,3]      #amplitude of different tonebursts
len_signal = 1000

def complete_signal(locs,amps,len_signal,nocycles=5,freq=1*10**5,samplefreq = 4*10**6):
    signal_start = np.zeros(len_signal)
    damage = damage_toneburst(nocycles=nocycles,freq=freq,samplefreq=samplefreq)
    i=0
    for loc in locs:
        signal_start[loc:loc+len(damage)] = amps[i]*damage
        i=i+1
    return signal_start


damage_toneburst_signal = damage_toneburst(nocycles,freq,samplefreq)
complete_signal = complete_signal(locs,amps,len_signal)

In [7]:
#adding noise to the test signals
damage_toneburst_noise = gaussian_noise(damage_toneburst_signal,mu,std)
complete_signal_noise = gaussian_noise(complete_signal,mu,std)

#plotting the signals along with the noisy signal
first_quad = go.Scatter(x=time, y=damage_toneburst_signal, name="Damage Toneburst Signal")
third_quad = go.Scatter(x= time, y= complete_signal, name="Complete Signal")
second_quad = go.Scatter(x= time, y= damage_toneburst_noise, name=" Damage Toneburst Signal with Noise")
fourth_quad = go.Scatter(x= time, y= complete_signal_noise, name=" Complete Signal with Noise")

fig = make_subplots(rows=2, cols=2, horizontal_spacing=0.01,shared_yaxes= True)

fig.add_trace(first_quad, row=1, col=1)
fig.add_trace(second_quad, row=1, col=2)
fig.add_trace(third_quad, row=2, col=1)
fig.add_trace(fourth_quad, row=2, col=2)

fig.update_layout(height=600, width=800, title_text=" Signal Visualization",legend=dict(
        x=1,
        y=.5,
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=12,
            color="white"
        ),
    ))
fig.layout.template = 'custom_dark'
fig.show()


# **LOW PASS FILTER**

In [8]:
def low_pass_filter(sig):
  # Define the low pass filter
  cutoff_freq = 3  # Hz
  from scipy import signal

  # Generate the time vector
  time = np.linspace(0, 2*np.pi, 1000)

  nyquist_freq = 0.5 * len(time) / time[-1]   #nyquist frequency = 1/2 * sampling frequency
  cutoff_norm = cutoff_freq / nyquist_freq
  b, a = signal.butter(4, cutoff_norm, 'lowpass')  #4 here is the steepness between cutoff and pass band.

  # Apply the low pass filter to the signal
  return signal.filtfilt(b, a, sig)   #forward backward filter implementation to remove phase distortions


# Plot the original and filtered signal
first_quad = go.Scatter(x=time, y=GaussianNoiseSignal, name="Gaussian Noise Sinosuidal Signal")
second_quad = go.Scatter(x= time, y= low_pass_filter(GaussianNoiseSignal), name="Low pass filter on gaussian sinosuidal noisy signal")
third_quad = go.Scatter(x= time, y= RandomNoiseSignal, name=" Random Noise Sinousuidal Signal")
fourth_quad = go.Scatter(x= time, y= low_pass_filter(RandomNoiseSignal), name=" Low pass filter on random sinosuidal noisy signal")
fifth_quad = go.Scatter(x= time, y= damage_toneburst_noise, name=" Damage Toneburst Signal with Gaussian Noise")
sixth_quad = go.Scatter(x= time, y= low_pass_filter(damage_toneburst_noise), name="Low pass filter on damage toneburst Gaussian Noisy Signal")
seventh_quad = go.Scatter(x= time, y= complete_signal_noise, name=" Complete Signal with Gaussian Noise")
eighth_quad = go.Scatter(x= time, y= low_pass_filter(complete_signal_noise), name="Low pass filter on  Complete Signal with gaussian Noise")

def custom_plot(head):
  fig = make_subplots(rows=4, cols=4, horizontal_spacing=0.01,shared_yaxes= True)

  fig.add_trace(first_quad, row=1, col=1)
  fig.add_trace(second_quad, row=1, col=2)
  fig.add_trace(third_quad, row=2, col=1)
  fig.add_trace(fourth_quad, row=2, col=2)
  fig.add_trace(fifth_quad, row=3, col=1)
  fig.add_trace(sixth_quad, row=3, col=2)
  fig.add_trace(seventh_quad, row=4, col=1)
  fig.add_trace(eighth_quad, row=4, col=2)

  fig.update_layout(height=600, width=1500, title_text=head,legend=dict(
          x=.5,
          y=.5,
          traceorder="normal",
          font=dict(
              family="sans-serif",
              size=12,
              color="white"
          ),
      ))
  fig.layout.template = 'custom_dark'
  fig.show()

custom_plot(" Low Pass filter Visualization")

# **BOX FILTER**

In [9]:
#GaussianNoiseSignal, RandomNoiseSignal,damage_toneburst_noise,complete_signal_noise

def box_filter(sig):
  # Define the box filter
  window_size = 20      #width of moving box
  box_filter = np.ones(window_size) / window_size

  # Apply the box filter to the signal
  filtered_signal = np.convolve(sig, box_filter, mode='same')
  return filtered_signal

# Plot the original and filtered signal
top = go.Scatter(x = time, y = OriginalSignal, name = "Original Signal")
first_quad = go.Scatter(x = time, y = GaussianNoiseSignal, name = "Gaussian Noise Sinosuidal Signal")
second_quad = go.Scatter(x = time, y = box_filter(GaussianNoiseSignal), name = "Box filter on gaussian sinosuidal noisy signal")
third_quad = go.Scatter(x = time, y = RandomNoiseSignal, name = " Random Noise Sinousuidal Signal")
fourth_quad = go.Scatter(x = time, y = box_filter(RandomNoiseSignal), name = "Box filter on random sinosuidal noisy signal")
fifth_quad = go.Scatter(x = time, y = damage_toneburst_noise, name = " Damage Toneburst Signal with Gaussian Noise")
sixth_quad = go.Scatter(x = time, y = box_filter(damage_toneburst_noise), name = "Box filter on damage toneburst Gaussian Noisy Signal")
seventh_quad = go.Scatter(x = time, y = complete_signal_noise, name = " Complete Signal with Gaussian Noise")
eighth_quad = go.Scatter(x = time, y = box_filter(complete_signal_noise), name = "Box filter on  Complete Signal with gaussian Noise")

custom_plot("Box filter Visualization")

# GAUSSIAN FILTER

In [10]:
from scipy.signal import gaussian, convolve

test_signal = complete_signal_noise

def gaussian_filter(sig):
  # Define the Gaussian filter
  filter_size = 15                #width of gaussian curve
  sigma = 10
  gaussian_filter = gaussian(filter_size, sigma)

  # Normalize the filter
  gaussian_filter /= sum(gaussian_filter)

  # Apply the Gaussian filter to the signal
  filtered_signal = convolve(sig, gaussian_filter, mode='same')
  return filtered_signal

# Plot the original and filtered signal
top = go.Scatter(x = time, y = OriginalSignal, name = "Original Signal")
first_quad = go.Scatter(x = time, y = GaussianNoiseSignal, name = "Gaussian Noise Sinosuidal Signal")
second_quad = go.Scatter(x = time, y = gaussian_filter(GaussianNoiseSignal), name = "gaussian_filter on gaussian sinosuidal noisy signal")
third_quad = go.Scatter(x = time, y = RandomNoiseSignal, name = " Random Noise Sinousuidal Signal")
fourth_quad = go.Scatter(x = time, y = gaussian_filter(RandomNoiseSignal), name = "gaussian_filter on random sinosuidal noisy signal")
fifth_quad = go.Scatter(x = time, y = damage_toneburst_noise, name = " Damage Toneburst Signal with Gaussian Noise")
sixth_quad = go.Scatter(x = time, y = gaussian_filter(damage_toneburst_noise), name = "gaussian_filter on damage toneburst Gaussian Noisy Signal")
seventh_quad = go.Scatter(x = time, y = complete_signal_noise, name = " Complete Signal with Gaussian Noise")
eighth_quad = go.Scatter(x = time, y = gaussian_filter(complete_signal_noise), name = "gaussian_filter on  Complete Signal with gaussian Noise")

custom_plot("Gaussian filter Visualization")

# SAVITZKY-GOLAY FILTER

In [11]:
test_signal = complete_signal_noise
time = np.linspace(0, 2*np.pi, 1000)

from scipy.signal import savgol_filter
window_length = 50
polyorder = 5
def savitzky_filter(sig):
  # Savitzky-Golay filter
  filtered_signal = savgol_filter(sig,window_length,polyorder)
  return filtered_signal

# Plot the original and filtered signal
top = go.Scatter(x=time, y=OriginalSignal, name="Original Signal")
first_quad = go.Scatter(x=time, y=GaussianNoiseSignal, name="Gaussian Noise Sinosuidal Signal")
second_quad = go.Scatter(x= time, y= savitzky_filter(GaussianNoiseSignal), name="savitzky_filter on gaussian sinosuidal noisy signal")
third_quad = go.Scatter(x= time, y= RandomNoiseSignal, name=" Random Noise Sinousuidal Signal")
fourth_quad = go.Scatter(x= time, y= savitzky_filter(RandomNoiseSignal), name="savitzky_filter on random sinosuidal noisy signal")
fifth_quad = go.Scatter(x= time, y= damage_toneburst_noise, name=" Damage Toneburst Signal with Gaussian Noise")
sixth_quad = go.Scatter(x= time, y= savitzky_filter(damage_toneburst_noise), name="savitzky_filter on damage toneburst Gaussian Noisy Signal")
seventh_quad = go.Scatter(x= time, y= complete_signal_noise, name=" Complete Signal with Gaussian Noise")
eighth_quad = go.Scatter(x= time, y= savitzky_filter(complete_signal_noise), name="savitzky_filter on  Complete Signal with gaussian Noise")

custom_plot(" Savitzky Golay filter Visualization")

# BANDPASS FILTER

In [12]:
from scipy.signal import butter, sosfiltfilt

test_signal = complete_signal_noise
time = np.linspace(0, 2*np.pi, 1000)

# Define the bandpass filter
def bandpass_filter(sig):

    fs = 1000  # Hz
    lowcut = 20  # Hz
    highcut = 50  # Hz
    order = 4

    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    sos = butter(order, [low, high], analog=False, btype='band', output='sos')
    
    # Apply the bandpass filter to the signal

    filtered_signal = sosfiltfilt(sos, sig)
    return filtered_signal

  
# Plot the original and filtered signal

top = go.Scatter(x=time, y=OriginalSignal, name="Original Signal")
first_quad = go.Scatter(x=time, y=GaussianNoiseSignal, name="Gaussian Noise Sinosuidal Signal")
second_quad = go.Scatter(x= time, y= bandpass_filter(GaussianNoiseSignal), name="bandpass_filter on gaussian sinosuidal noisy signal")
third_quad = go.Scatter(x= time, y= RandomNoiseSignal, name=" Random Noise Sinousuidal Signal")
fourth_quad = go.Scatter(x= time, y= bandpass_filter(RandomNoiseSignal), name="bandpass_filter on random sinosuidal noisy signal")
fifth_quad = go.Scatter(x= time, y= damage_toneburst_noise, name=" Damage Toneburst Signal with Gaussian Noise")
sixth_quad = go.Scatter(x= time, y= bandpass_filter(damage_toneburst_noise), name="bandpass_filter on damage toneburst Gaussian Noisy Signal")
seventh_quad = go.Scatter(x= time, y= complete_signal_noise, name=" Complete Signal with Gaussian Noise")
eighth_quad = go.Scatter(x= time, y= bandpass_filter(complete_signal_noise), name="bandpass_filter on  Complete Signal with gaussian Noise")

custom_plot("Bandpass filter Visualization")

In [13]:
def custom_plot_bulk(x):
  fig = make_subplots(rows = 1, cols = 6, horizontal_spacing = 0.01, shared_yaxes = True, subplot_titles = ("Plot 1", "Plot 2", "Plot 3", "Plot 4","Plot 5","Plot 6"))

  fig.add_trace(r1_c1, row = 1, col = 1)
  fig.add_trace(r1_c2, row = 1, col = 2)
  fig.add_trace(r1_c3, row = 1, col = 3)
  fig.add_trace(r1_c4, row = 1, col = 4)
  fig.add_trace(r1_c5, row = 1, col = 5)
  fig.add_trace(r1_c6, row = 1, col = 6)

  fig.update_layout(height = 300, width = 2000, title_text = x,legend = dict(
          x = 1.1,
          y = .5,
          traceorder = "normal",
          font = dict(
              family = "sans-serif",
              size = 12,
              color = "white"
          ),
      ))

  names = {'Plot 1':'Unflitered Signal','Plot 2':'Low pass filter', 'Plot 3':'Box Filter', 'Plot 4':'Gaussian Filter', 'Plot 5':'Savitzky Golay filter','Plot 6':'Bandpass filter'}
  fig.for_each_annotation(lambda a: a.update(text = names[a.text]))
  
  fig.layout.template = 'custom_dark'
  fig.show()

# Plot the original and filtered signal

Annotations  = ["Gaussian Noise Sinusoidal Signal", "Random Noise Sinusoidal Signal","Damage Toneburst Signal with Gaussian Noise","Complete Signal with Gaussian Noise"]
noise_list = [GaussianNoiseSignal,RandomNoiseSignal,damage_toneburst_noise,complete_signal_noise]
for z in range(len(noise_list)):
  r1_c1 = go.Scatter(x = time, y = noise_list[z], name = f"{Annotations[z]}")
  r1_c2 = go.Scatter(x = time, y = low_pass_filter(noise_list[z]), name = f"Low pass filter on {Annotations[z]}")
  r1_c3 = go.Scatter(x = time, y = box_filter(noise_list[z]), name = f"Box filter on {Annotations[z]}")
  r1_c4 = go.Scatter(x = time, y = gaussian_filter(noise_list[z]), name = f"gaussian_filter on {Annotations[z]}")
  r1_c5 = go.Scatter(x = time, y = savitzky_filter(noise_list[z]), name = f"savitzky_filter on {Annotations[z]}")
  r1_c6 = go.Scatter(x = time, y = bandpass_filter(noise_list[z]), name = f"bandpass_filter on {Annotations[z]}")

  custom_plot_bulk(f"<b>{Annotations[z]}</b>")


In [14]:
def visualize(path: str):
	raw = wave.open(path)
	signal = raw.readframes(-1)
	signal = np.frombuffer(signal, dtype ="int16")
	f_rate = raw.getframerate()

	time = np.linspace(0,len(signal) / f_rate,num = len(signal))
	plt.plot(time, signal) 
  #return signal,time

#OriginalSignal, time = visualize()
#custom_plot_bulk(OriginalSignal)



### **Question 2**

Generate a signals which are mix of a range of low-frequencies and a range of high frequencies and add noise to it.

In [15]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

def four_plot_signal(fs , duration , low_freq , high_freq):
    # fs is sampling rate
    # duration duartion in seconds

    t = np.linspace(0, duration, duration * fs, endpoint=False)

    low_signal = np.sin(2 * np.pi * low_freq * t)
    high_signal = np.sin(2 * np.pi * high_freq * t)

    # Add noise to the signal
    noise = np.random.normal(0, 1, len(t))
    signals = low_signal + high_signal + noise

    first_line = go.Scatter(x=t, y=low_signal+high_signal, name="Original Signal")
    second_line = go.Scatter(x= t, y= signals, name="Noisy Signal")

    fig = make_subplots(rows=2, cols=1, vertical_spacing=0.10)

    fig.add_trace(first_line, row=1, col=1)
    fig.add_trace(second_line, row=2, col=1)

    fig.update_layout(height=600, width=800, title_text=" Signal Visualization")
    fig.layout.template = 'custom_dark'
    fig.show()

fs = 1000 #sampling frequency
duration = 1
low_freq = 5
high_freq = 100

four_plot_signal(fs , duration , low_freq , high_freq)

In [16]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

def four_plot_signal(fs , duration , low_freq , high_freq):
    # fs is sampling rate
    # duration duartion in seconds

    t = np.linspace(0, duration, duration * fs, endpoint=False)

    low_signal = np.sin(2 * np.pi * low_freq * t)
    high_signal = np.sin(2 * np.pi * high_freq * t)

    # Add noise to the signal
    noise = np.random.normal(0, 1, len(t))
    signals = low_signal + high_signal + noise

    freqs = np.fft.fftfreq(len(signals), 1 / fs)
    fft = np.fft.fft(signals)


    cutoff_freq = 10  # Cutoff frequency (in Hz) { cutoff frequency = low_freq*2}
    #b, a = signal.butter(4, cutoff_freq / (fs / 2), 'low')
    #filtered_signal = signal.filtfilt(b, a, signals)
    fft[np.abs(freqs) > cutoff_freq] = 0  # Filter out high frequencies
    # Inverse Fourier transform
    filtered_signal = np.fft.ifft(fft).real

    # Plot the signals
    first_line = go.Scatter(x=t, y = signals, name="Original Signal")
    second_line = go.Scatter(x= freqs[:len(freqs)//2], y= np.abs(fft[:len(freqs)//2]), name="Fourier Transform")
    third_line = go.Scatter(x= t, y= filtered_signal, name="Filtered Signal")
    forth_line = go.Scatter(x= t, y= low_signal, name="Low Signal")

    fig = make_subplots(rows=4, cols=1, vertical_spacing=0.10)

    fig.add_trace(first_line, row=1, col=1)
    fig.add_trace(second_line, row=2, col=1)
    fig.add_trace(third_line, row=3, col=1)
    fig.add_trace(forth_line, row=4, col=1)


    fig.update_layout(height=600, width=800, title_text=" Signal Visualization")
    fig.layout.template = 'custom_dark'
    fig.show()

   

fs = 1000 #sampling frequency
duration = 1
low_freq = 5
high_freq = 100

four_plot_signal(fs , duration , low_freq , high_freq)